In [20]:
import pandas as pd
import numpy as np

In [22]:
%load_ext rpy2.ipython

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,name_length,title,title_group,family_size,is_alone,ticket_group_size,cabin_score,name_word_count,Age_missing,Cabin_missing,log_Fare,log_fare_per_person,log_age_fare_ratio
0,1,1,female,35.00,1,0,S,47,Mrs,Mrs,2,0,2,9.173136,7,False,False,4.510860,3.828641,0.328504
1,0,2,male,19.00,0,0,S,31,Mr,Mr,1,1,1,9.064721,4,False,True,2.442347,2.442347,1.033015
2,0,2,male,34.00,0,0,S,28,Mr,Mr,1,1,1,2.131047,4,False,True,2.639057,2.639057,1.285198
3,1,3,female,4.00,0,2,S,35,Miss,Miss,3,0,1,1.148368,4,False,True,3.136581,3.136581,0.166879
4,1,2,female,21.00,0,0,S,17,Miss,Miss,1,1,1,3.922440,3,False,True,2.442347,2.442347,1.098612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,1,2,male,NaN,0,0,C,26,Mr,Mr,1,1,1,4.019309,5,True,True,2.698841,2.698808,0.000000
664,1,3,female,0.75,2,1,C,29,Miss,Miss,4,0,4,5.968428,4,False,True,3.008565,1.760441,0.038205
665,0,3,female,48.00,1,3,S,39,Mrs,Mrs,5,0,4,1.312449,6,False,True,3.566005,2.261138,0.873952
666,0,1,male,47.00,0,0,S,17,Mr,Mr,1,1,1,7.325721,4,False,False,3.676301,3.676301,0.797858


In [54]:
feature_names = [f"x{i}" for i in range(train_X.shape[1])]  # 临时方案
X = pd.DataFrame(train_X, columns=feature_names)
y = pd.Series(train_y).astype(int)

In [55]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

# 你的数据：y = train_y (0/1), X = train_X (DataFrame)
X = train_X.copy()
y = pd.Series(train_y).astype(int)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def build_preprocess(X_df: pd.DataFrame) -> ColumnTransformer:
    """根据传入的X子集动态构建预处理器，避免列名不匹配。"""
    num_cols = X_df.select_dtypes(include=["number", "bool"]).columns.tolist()
    cat_cols = X_df.select_dtypes(exclude=["number", "bool"]).columns.tolist()

    numeric_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    categorical_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    return ColumnTransformer([
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols)
    ], remainder="drop")

def cv_score_for_features(feature_list, scoring="roc_auc"):
    X_sub = X[feature_list].copy()
    preprocess = build_preprocess(X_sub)

    model = LogisticRegression(max_iter=5000, solver="liblinear")
    pipe = Pipeline([("prep", preprocess), ("model", model)])

    scores = cross_val_score(pipe, X_sub, y, cv=cv, scoring=scoring, error_score="raise")
    return scores.mean()

def forward_selection(max_features=12, scoring="roc_auc", min_improve=1e-3):
    remaining = list(X.columns)
    selected = []
    best_score = -np.inf
    history = []

    for _ in range(max_features):
        step_best_feat = None
        step_best_score = best_score

        for feat in remaining:
            s = cv_score_for_features(selected + [feat], scoring=scoring)
            if s > step_best_score:
                step_best_score = s
                step_best_feat = feat

        if step_best_feat is None or (step_best_score - best_score) < min_improve:
            break

        selected.append(step_best_feat)
        remaining.remove(step_best_feat)
        best_score = step_best_score
        history.append((len(selected), step_best_feat, best_score))
        print(f"Step {len(selected)}: + {step_best_feat:20s} | CV {scoring} = {best_score:.4f}")

    return selected, pd.DataFrame(history, columns=["step", "added_feature", f"cv_{scoring}"])

selected_stepwise, step_hist = forward_selection(max_features=12, scoring="roc_auc", min_improve=1e-3)
selected_stepwise, step_hist


AttributeError: 'numpy.ndarray' object has no attribute 'columns'